<a href="https://colab.research.google.com/github/mjiii25/posco-academy/blob/main/Big-Data/Battery-Project/hypothesis-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **가설 검정**

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [8]:
import pandas as pd
import numpy as np

import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

### **데이터 구성하기**

In [3]:
df_raw_process = pd.read_csv('/content/mydrive/MyDrive/posco-big-data/bat_process.csv', encoding = 'euc-kr')


print(df_raw_process.shape)
df_raw_process.head()

(43754, 109)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (97,101,102,103) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,sa3_box_row,sa3_box_dan,socv3_ocv,ocv2_deltaocv,ocv1_deltaocv,m1_thick,m1_voltage,m1_res_ac,m1_mv,m1_voltage_an
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,37연,04단,3759.3,5,6.8,4718,37583,94,565,164
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,37연,04단,3758.4,5,6.8,4718,37574,94,565,73
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,37연,04단,3759,5.1,6.9,4700,37579,91,585,-57
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,37연,04단,3759.5,5.1,6.9,4688,37583,93,606,99
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,37연,04단,3758.2,5,6.8,4696,37570,91,605,36


In [4]:
df_raw_tat = pd.read_csv('/content/mydrive/MyDrive/posco-big-data/bat_tat.csv', encoding = 'euc-kr')


print(df_raw_tat.shape)
df_raw_tat.head()

(43754, 25)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,lot_id,tray_id,dt_start,cell_id,rta1_tat,hta1_tat,rta2_tat,ocv1_tat,c1_tat,dc1_tat,...,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,217066,86461,5732,119,3556,404,...,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,217066,86461,5732,119,3556,404,...,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,217066,86461,5732,118,3556,405,...,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,217066,86461,5732,118,3556,405,...,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,217066,86461,5732,119,3556,404,...,118,290,290,53500,5,355357,5,215150,5,207532


In [ ]:
# df_raw.loc[df_raw['judge'] == '불량', 'judge'] = 1
# df_raw.loc[df_raw['judge'] == '양품', 'judge'] = 0

In [5]:
df_raw = pd.merge(df_raw_process, df_raw_tat)

print(df_raw.shape)
df_raw.head()

(43754, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532


In [6]:
df_raw.replace('.', np.NaN, inplace=True)
df_raw.replace(' ', np.NaN, inplace=True)

In [7]:
df_raw.dropna(inplace = True)

### **(21열, 22열)이랑 (6열, 9열, 18열) 나누기**

In [42]:
# 22열, 21열

df_raw22 = df_raw[df_raw['c4_box_col'] == '22열']
df_raw21 = df_raw[df_raw['c4_box_col'] == '21열']

df_raw_top = pd.concat([df_raw22, df_raw21], axis = 0)

print(df_raw_top.shape)
df_raw_top.head()

(4988, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
0,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10747,양품,10,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
1,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10748,양품,7,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
2,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10749,양품,19,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532
3,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10750,양품,24,08열,33연,08단,03열,...,118,290,290,53501,5,355357,5,215150,5,207532
4,LOT-100,TRAY-1023,12-01-2021 오전 10:01:25,CELL-10751,양품,17,08열,33연,08단,03열,...,118,290,290,53500,5,355357,5,215150,5,207532


In [46]:
# 9열, 4열, 6열

df_raw9 = df_raw[df_raw['c4_box_col'] == '09열']
df_raw18 = df_raw[df_raw['c4_box_col'] == '04열']
df_raw6 = df_raw[df_raw['c4_box_col'] == '06열']

df_raw_bot = pd.concat([df_raw9, df_raw18], axis = 0)
df_raw_bot = pd.concat([df_raw_bot, df_raw6], axis = 0)

print(df_raw_bot.shape)
df_raw_bot.head()

(4876, 130)


,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,hta1_box_col,...,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
396,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10175,양품,1,07열,05연,01단,01열,...,118,290,290,54522,5,355271,5,215550,5,208516
397,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10176,양품,7,07열,05연,01단,01열,...,118,290,290,54522,5,355271,5,215550,5,208516
398,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10177,양품,24,07열,05연,01단,01열,...,118,290,290,54522,5,355271,5,215550,5,208516
399,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10178,양품,5,07열,05연,01단,01열,...,118,290,290,54522,5,355271,5,215550,5,208516
400,LOT-100,TRAY-1008,12-01-2021 오전 6:36:25,CELL-10179,양품,14,07열,05연,01단,01열,...,118,290,290,54522,5,355271,5,215550,5,208516


In [47]:
df_raw_total = pd.concat([df_raw_top, df_raw_bot], axis = 0)
df_raw_total = df_raw_total.reset_index()

print(df_raw_total.shape)
df_raw_total.tail()

(9864, 131)


,index,lot_id,tray_id,dt_start,cell_id,judge,rta1_cell_no,rta1_box_col,rta1_box_row,rta1_box_dan,...,ocv2_tat,pg1_tat,pc1_tat,sa1_tat,socv1_tat,sa2_tat,socv2_tat,sa3_tat,socv3_tat,sa4_tat
9859,43428,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53553,양품,12,07열,58연,09단,...,118,290,291,42934,5,345063,5,217121,5,158024
9860,43429,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53554,양품,9,07열,58연,09단,...,118,290,291,42934,5,345063,5,217121,5,158024
9861,43430,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-53555,양품,24,07열,58연,09단,...,118,290,291,42934,5,345063,5,217121,5,158024
9862,43431,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-54255,양품,1,07열,58연,09단,...,118,290,290,42934,6,345063,5,217121,5,158024
9863,43432,LOT-144,TRAY-2773,12-17-2021 오후 2:36:45,CELL-54256,양품,2,07열,58연,09단,...,118,290,290,42934,6,345063,5,217121,5,158024


In [49]:
df_raw_total.loc[df_raw_total['c4_box_col'] == '04열', 'c4_box_col'] = 0
df_raw_total.loc[df_raw_total['c4_box_col'] == '06열', 'c4_box_col'] = 0
df_raw_total.loc[df_raw_total['c4_box_col'] == '09열', 'c4_box_col'] = 0

df_raw_total.loc[df_raw_total['c4_box_col'] == '21열', 'c4_box_col'] = 1
df_raw_total.loc[df_raw_total['c4_box_col'] == '22열', 'c4_box_col'] = 1

In [51]:
df_raw_total['c4_box_col']

0       1
1       1
2       1
3       1
4       1
       ..
9859    0
9860    0
9861    0
9862    0
9863    0
Name: c4_box_col, Length: 9864, dtype: object

#### **카이제곱검정**


In [55]:
ct = pd.crosstab(df_raw_total['c4_box_col'], df_raw_total['judge'])
ct

judge,불량,양품
c4_box_col,,
0,46,4830
1,421,4567


In [57]:
stat, pval, df, expected = stats.chi2_contingency(observed = ct)

print('***** 카이제곱검정 *****')
print('H0 : 설비에 따라 양품불량개수의 차이가 없다.')
print('H1 : 설비에 따라 양품불량개수의 차이가 있다.')
print('')
print('통계량 : ', stat.round(3))
print('pvalue : ', pval.round(3))

***** 카이제곱검정 *****
H0 : 설비에 따라 양품불량개수의 차이가 없다.
H1 : 설비에 따라 양품불량개수의 차이가 있다.

통계량 :  305.593
pvalue :  0.0


- 검정통계량 305.593에 대한 유의확률 = 0.0 < 0.05이므로 귀무가설을 기각한다.
- 즉, 설비에 따라 양품불량개수의 차이가 있다.

### **ANOVA 검정**


In [54]:
# ANOVA 실행

print('***** ANOVA분석 *****')
print('H0 : 설비에 따라 온도에 차이가 없다.')
print('H1 : 설비에 따라 온도에 차이가 있다.')
print('')
model = ols('c4_temp_avg ~ C(c4_box_col)', df_raw_total).fit()
anova_lm(model)

***** ANOVA분석 *****
H0 : 설비에 따라 온도에 차이가 없다.
H1 : 설비에 따라 온도에 차이가 있다.



,df,sum_sq,mean_sq,F,PR(>F)
C(c4_box_col),1.0,7.271406e+05,727140.612131,3634.647688,0.0
Residual,9862.0,1.972973e+06,200.058073,NaN,NaN


- 검정통계량 F = 3634.647688에 대한 유의확률 0.0 < 0.05이므로 귀무가설을 기각한다.
- 설비에 따라 온도에 차이가 있다.

### **2 sample t-test**



In [58]:
# 정규성 검정

statistic1, p1 = stats.shapiro(df_raw_total.loc[df_raw_total['judge'] == '불량', 'c4_temp_avg'])
print("----------- 불량에 대한 정규성 검정 -----------")
print('H0 : 정규성을 만족한다.')
print('H1 : 정규성을 만족하지 않는다.')
print('')
print('통계량 : ', statistic1)
print('pvalue : ', p1)
print('')

statistic2, p2 = stats.shapiro(df_raw_total.loc[df_raw_total['judge'] == '양품', 'c4_temp_avg'])
print("----------- 양품에 대한 정규성 검정 -----------")
print('H0 : 정규성을 만족한다.')
print('H1 : 정규성을 만족하지 않는다.')
print('')
print('통계량 : ', statistic2)
print('pvalue : ', p2)
print('')

----------- 불량에 대한 정규성 검정 -----------
H0 : 정규성을 만족한다.
H1 : 정규성을 만족하지 않는다.

통계량 :  0.8832783102989197
pvalue :  2.611841188904814e-18

----------- 양품에 대한 정규성 검정 -----------
H0 : 정규성을 만족한다.
H1 : 정규성을 만족하지 않는다.

통계량 :  0.9911037683486938
pvalue :  8.634443905938565e-24



/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


- 불량과 양품 개수에 따른 온도의 유의확률 2.868, 6.133 > 0.05이므로 귀무가설을 채택한다.
- 두 집단 모두 정규성을 만족한다.

In [ ]:
# 등분산성 검정

from scipy.stats import bartlett

statistic, p = stats.bartlett(df_raw.loc[df_raw['judge'] == '불량', 'c4_temp_avg'], 
                              df_raw.loc[df_raw['judge'] == '양품', 'c4_temp_avg'])
print("----------- 불량과 양품에 대한 등분산성 검정 -----------")
print('H0 : 등분산성을 만족한다.')
print('H1 : 등분산성을 만족하지 않는다.')
print('')
print('통계량 : ', statistic1)
print('pvalue : ', p)
print('')

----------- 불량과 양품에 대한 등분산성 검정 -----------
H0 : 등분산성을 만족한다.
H1 : 등분산성을 만족하지 않는다.

통계량 :  0.9536409974098206
pvalue :  3.648453010322984e-143



- 등분산성에 대한 검정통계량의 유의확률 3.648 > 0.05이므로 귀무가설을 채택한다.
- 등분산성이 만족된다.

In [59]:
# 대응표본 t-검정

statistic, p = stats.ttest_ind(df_raw_total.loc[df_raw_total['judge'] == '불량', 'c4_temp_avg'], 
                              df_raw_total.loc[df_raw_total['judge'] == '양품', 'c4_temp_avg'])
print("----------- 2-Sample t-test -----------")
print('H0 : 양품과 불량 사이에 온도 차이가 없다.')
print('H1 : 양품과 불량 사이에 온도 차이가 있다.')
print('')
print('통계량 : ', statistic)
print('pvalue : ', p)
print('')

----------- 2-Sample t-test -----------
H0 : 양품과 불량 사이에 온도 차이가 없다.
H1 : 양품과 불량 사이에 온도 차이가 있다.

통계량 :  -27.944668656171597
pvalue :  1.902627052174875e-165



- 양품불량개수에 대한 유의확률 0.000 < 0.05이므로 귀무가설을 기각한다.
- 양품과 불량 사이에 온도 차이가 존재한다.

In [63]:
# 대응표본 t-검정

statistic, p = stats.ttest_ind(df_raw_total.loc[df_raw_total['judge'] == '불량', 'c4_voltage_avg'], 
                              df_raw_total.loc[df_raw_total['judge'] == '양품', 'c4_voltage_avg'])
print("----------- 2-Sample t-test -----------")
print('H0 : 양품과 불량 사이에 전압 차이가 없다.')
print('H1 : 양품과 불량 사이에 전압 차이가 있다.')
print('')
print('통계량 : ', statistic)
print('pvalue : ', p)
print('')

----------- 2-Sample t-test -----------
H0 : 양품과 불량 사이에 전압 차이가 없다.
H1 : 양품과 불량 사이에 전압 차이가 있다.

통계량 :  13.263307232539386
pvalue :  8.276636383150192e-40



- 양품불량개수에 대한 유의확률 0.000 < 0.05이므로 귀무가설을 기각한다.
- 양품과 불량 사이에 전압 차이가 존재한다.

In [62]:
# 대응표본 t-검정

statistic, p = stats.ttest_ind(df_raw_total.loc[df_raw_total['judge'] == '불량', 'c4_curr_end'], 
                              df_raw_total.loc[df_raw_total['judge'] == '양품', 'c4_curr_end'])
print("----------- 2-Sample t-test -----------")
print('H0 : 양품과 불량 사이에 전류 차이가 없다.')
print('H1 : 양품과 불량 사이에 전류 차이가 있다.')
print('')
print('통계량 : ', statistic)
print('pvalue : ', p)
print('')

----------- 2-Sample t-test -----------
H0 : 양품과 불량 사이에 전류 차이가 없다.
H1 : 양품과 불량 사이에 전류 차이가 있다.

통계량 :  0.9803863850638974
pvalue :  0.32691947699851



- 양품불량개수에 대한 유의확률 0.3269 > 0.05이므로 귀무가설을 채택한다.
- 양품과 불량 사이에 전류 차이가 존재하지 않는다.

**가설검정 결론**

- 중요한 공정조건 : 온도, 전압, 전류

In [ ]:
df_bad_top2 = [df_bad['c1_box_col'].value_counts().sort_values(ascending = False)[0],
               df_bad['c1_box_col'].value_counts().sort_values(ascending = False)[1]]

In [ ]:
df_bad_bot3 = [df_bad['c1_box_col'].value_counts().sort_values(ascending = False)[19],
               df_bad['c1_box_col'].value_counts().sort_values(ascending = False)[21]]

In [ ]:
df_bad_rank = pd.DataFrame()
df_bad_rank['top2'] = df_bad_top2
df_bad_rank['mid2'] = df_bad_mid2
df_bad_rank['bot2'] = df_bad_bot2

print(df_bad_rank)
df_bad_rank

   top2  mid2  bot2
0   291    64    20
1   136    64    14


,top2,mid2,bot2
0,291,64,20
1,136,64,14


In [ ]:
corr, pval = stats.pearsonr(df_bad_rank['top2'], df_bad_rank['bot2'])

print("------- Correlation Analysis -------")
print("corr : {0:0.3f}".format(corr))
print("p-value : {0:0.3f}".format(pval))

------- Correlation Analysis -------
corr : 1.000
p-value : 1.000
